In [6]:
import numpy as np
import nibabel as nib

In [1]:

#raise error if data type doesn't match 4D array -> function expects preprocessed BOLD fMRI file 
import numpy as np

# Define the dimensions of the 4D array (shape: (x, y, z, t))
x_dim = 10  # Number of rows in each 3D slice
y_dim = 10  # Number of columns in each 3D slice
z_dim = 5   # Number of slices
t_dim = 20  # Number of time points

# Generate a random 4D array
random_4d_array = np.random.rand(x_dim, y_dim, z_dim, t_dim)


In [56]:
##Create fake BOLD signal file

import os
import numpy as np
import nibabel as nib

# Atlas dimensions
x_dim = 91
y_dim = 109
z_dim = 91

# Number of time points
t_dim = 200

# Number of brain areas in the atlas
brain_par = 360

# Create the "Subjects" directory if it doesn't exist
subjects_directory = "Subjects"
if not os.path.exists(subjects_directory):
    os.makedirs(subjects_directory)

# Generate and save 20 different fake BOLD signal files, one for each subject
num_subjects = 20
for subject_idx in range(1, num_subjects + 1):
    # Generate random BOLD signal data
    random_bold_data = np.random.randn(x_dim, y_dim, z_dim, t_dim)
    
    # Create a NIfTI image header
    header = nib.Nifti1Header()
    header.set_data_dtype(np.float32)
    
    # Save BOLD signal data as a NIfTI file
    bold_nifti = nib.Nifti1Image(random_bold_data, np.eye(4), header=header)
    
    # Save the NIfTI file without compression in the "Subjects" folder
    file_name = f"fake_BOLD.nii"
    subject_directory = os.path.join(subjects_directory, f"Subject_{subject_idx}")
    if not os.path.exists(subject_directory):
        os.makedirs(subject_directory)
    file_path = os.path.join(subject_directory, file_name)
    nib.save(bold_nifti, file_path)
    print(f"Saved {file_path}")



Saved Subjects/Subject_1/fake_BOLD.nii
Saved Subjects/Subject_2/fake_BOLD.nii
Saved Subjects/Subject_3/fake_BOLD.nii
Saved Subjects/Subject_4/fake_BOLD.nii
Saved Subjects/Subject_5/fake_BOLD.nii
Saved Subjects/Subject_6/fake_BOLD.nii
Saved Subjects/Subject_7/fake_BOLD.nii
Saved Subjects/Subject_8/fake_BOLD.nii
Saved Subjects/Subject_9/fake_BOLD.nii
Saved Subjects/Subject_10/fake_BOLD.nii
Saved Subjects/Subject_11/fake_BOLD.nii
Saved Subjects/Subject_12/fake_BOLD.nii
Saved Subjects/Subject_13/fake_BOLD.nii
Saved Subjects/Subject_14/fake_BOLD.nii
Saved Subjects/Subject_15/fake_BOLD.nii
Saved Subjects/Subject_16/fake_BOLD.nii
Saved Subjects/Subject_17/fake_BOLD.nii
Saved Subjects/Subject_18/fake_BOLD.nii
Saved Subjects/Subject_19/fake_BOLD.nii
Saved Subjects/Subject_20/fake_BOLD.nii


In [57]:
import os
import glob
import nibabel as nib
import numpy as np

# Directory where subject folders are located
name_session = "/Users/neuro-240/Documents/Subjects"
subjects_directory = name_session

# Search for all subject folders
subject_folders = glob.glob(os.path.join(subjects_directory, "Subject_*"))

# Loop through each subject's folder
for subject_folder in subject_folders:
    bold_file_path = os.path.join(subject_folder, "fake_BOLD.nii")
    if os.path.exists(bold_file_path):
        bold_nifti = nib.load(bold_file_path)
        bold_data = bold_nifti.get_fdata()
        
        # Process the bold_data as needed for each subject
        r, c, d, t = bold_data.shape
        averaged_bold_seq = np.zeros((t, brain_par))
        
        print(f"Loaded BOLD file: {bold_file_path}, Shape: {bold_data.shape}")
    else:
        print(f"No BOLD file found for {subject_folder}")


Loaded BOLD file: /Users/neuro-240/Documents/Subjects/Subject_12/fake_BOLD.nii, Shape: (91, 109, 91, 200)
Loaded BOLD file: /Users/neuro-240/Documents/Subjects/Subject_15/fake_BOLD.nii, Shape: (91, 109, 91, 200)
Loaded BOLD file: /Users/neuro-240/Documents/Subjects/Subject_14/fake_BOLD.nii, Shape: (91, 109, 91, 200)
Loaded BOLD file: /Users/neuro-240/Documents/Subjects/Subject_13/fake_BOLD.nii, Shape: (91, 109, 91, 200)
Loaded BOLD file: /Users/neuro-240/Documents/Subjects/Subject_4/fake_BOLD.nii, Shape: (91, 109, 91, 200)
Loaded BOLD file: /Users/neuro-240/Documents/Subjects/Subject_3/fake_BOLD.nii, Shape: (91, 109, 91, 200)
Loaded BOLD file: /Users/neuro-240/Documents/Subjects/Subject_2/fake_BOLD.nii, Shape: (91, 109, 91, 200)
Loaded BOLD file: /Users/neuro-240/Documents/Subjects/Subject_5/fake_BOLD.nii, Shape: (91, 109, 91, 200)
Loaded BOLD file: /Users/neuro-240/Documents/Subjects/Subject_16/fake_BOLD.nii, Shape: (91, 109, 91, 200)
Loaded BOLD file: /Users/neuro-240/Documents/Subje

In [2]:

# Load BOLD resting state data
folder_bold = f"{name_session}/4/NIFTI/"
bold_file = nib.load(f"{folder_bold}BOLDrestingCAP_pp_final.nii")
bold_sequence = bold_file.get_fdata()
r, c, d, t = bold_sequence.shape
averaged_bold_seq = np.zeros((t, brain_par))

# Load atlas registered to the BOLD file
atlas_file = nib.load(f"{folder_bold}atlas_reg.nii.gz")
atlas = atlas_file.get_fdata()

# Averaging Bold signal
for frame in range(t):
    bold_frame = bold_sequence[:, :, :, frame]
    for b in range(1, brain_par + 1):
        temp = np.zeros(2)
        for all_d in range(d):
            indices_x, indices_y = np.where(atlas[:, :, all_d] == b)
            for kk in range(len(indices_x)):
                temp = np.append(temp, bold_frame[indices_x[kk], indices_y[kk], all_d])
        temp = temp[temp != 0]
        averaged_bold_seq[frame, b - 1] = np.mean(temp)

# Remove mean from the already averaged bold sequences
mean_sig = np.mean(averaged_bold_seq, axis=1)
for b in range(brain_par):
    averaged_bold_seq[:, b] -= mean_sig

if use_deconv:
    TR = 1.4
    thr = 1
    event_lag_max = round(10 / TR)
    averaged_bold_seq, onset, hrf, event_lag, PARA = wgr_deconv_canonhrf_par(
        averaged_bold_seq, thr, event_lag_max, TR
    )
